In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')
from sklearn.preprocessing import MinMaxScaler


In [2]:
PATH ='C:/Users/ckdrj/Google 드라이브/baseball/data/total/'
hiter16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀타자_2016.csv")
pitcher16 = pd.read_csv(PATH+"2020빅콘테스트_스포츠투아이_제공데이터_팀투수_2016.csv")

In [3]:
TEAM_ID ='SS'
pitcher16 = pitcher16[pitcher16.T_ID==TEAM_ID].reset_index(drop=True)
pitcher16['index_date']=pd.date_range('2016-04-01', periods=144, freq='D')
pitcher16 = pitcher16.set_index('index_date')
pitcher16["ERA"] = 27*pitcher16.ER/pitcher16.INN2
pitcher16["FIP"] = ((13*pitcher16.HR+3*(pitcher16.BB+pitcher16.HP-pitcher16.IB)-2*pitcher16.KK)/(pitcher16.INN2/3)) +3.2
pitcher16["BABIP"] = (pitcher16.HIT-pitcher16.HR)/(pitcher16.AB-pitcher16.KK-pitcher16.HR+pitcher16.SF)
pitcher16 = pitcher16[['T_ID','ERA','FIP','BABIP','KK','R']]

In [4]:
pitcher16.head()

,T_ID,ERA,FIP,BABIP,KK,R
index_date,,,,,,
2016-04-01,SS,4.0,6.755556,0.222222,3,5
2016-04-02,SS,6.0,4.644444,0.354839,6,6
2016-04-03,SS,9.0,6.325000,0.370370,6,8
2016-04-04,SS,6.0,6.866667,0.148148,6,6
2016-04-05,SS,1.0,2.533333,0.190476,9,1


In [5]:
def create_dataset(signal_data, look_back=1):
    dataX, dataY = [], []
    for i in range(len(signal_data)-look_back):
        dataX.append(signal_data[i:(i+look_back), 0])
        dataY.append(signal_data[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [6]:
def prepro(col, look_back):
    signal_data = pitcher16[[col]]
    # 데이터 전처리
    scaler = MinMaxScaler(feature_range=(0, 1))
    signal_data = scaler.fit_transform(signal_data)

    # 데이터 분리
    train = signal_data[0:120]
    #val = signal_data[800:1200]
    test = signal_data[120:]

    # 데이터셋 생성
    x_train, y_train = create_dataset(train, look_back)
    #x_val, y_val = create_dataset(val, look_back)
    x_test, y_test = create_dataset(test, look_back)

    x_train = x_train.reshape(x_train.shape[0],look_back,1) #(size, timestamp,feature)
    x_test = x_test.reshape(x_test.shape[0],look_back,1)
    
    return x_train, x_test, y_train, y_test,scaler

In [33]:
x_train, x_test, y_train, y_test,scaler = prepro("R",10)

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout
from keras.callbacks import EarlyStopping
look_back=10
model = Sequential()
model.add(LSTM(20, input_shape=(look_back, 1))) #(timestep,feature)
model.add(Dropout(0.1))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam')

early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model.fit(x_train, y_train, epochs=100,
          batch_size=5, verbose=1, callbacks=[early_stop])

Epoch 1/100
110/110 [==============================] - 1s 10ms/step - loss: 0.1184
Epoch 2/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0556
Epoch 3/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0560
Epoch 4/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0542
Epoch 5/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0552
Epoch 6/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0567
Epoch 7/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0551
Epoch 00007: early stopping


In [35]:
model.evaluate(x_test,y_test,batch_size=5)

14/14 [==============================] - 0s 16ms/step


0.029328403728348867

In [36]:
pre =model.predict(x_test)
scaler.inverse_transform(pre)


array([[5.4329987],
       [5.2708864],
       [5.3393507],
       [5.613637 ],
       [5.6419525],
       [5.9316688],
       [5.867963 ],
       [5.704913 ],
       [5.8965716],
       [5.771264 ],
       [6.045732 ],
       [5.9861526],
       [5.8611283],
       [5.578115 ]], dtype=float32)

In [62]:
scaler.inverse_transform(pre).mean()

5.7101665

In [37]:
scaler.inverse_transform(y_test.reshape(14,1))

array([[ 5.],
       [ 6.],
       [10.],
       [ 3.],
       [ 9.],
       [ 5.],
       [ 3.],
       [10.],
       [ 5.],
       [10.],
       [ 4.],
       [ 4.],
       [ 3.],
       [ 7.]])

In [63]:
scaler.inverse_transform(y_test.reshape(14,1)).mean()

6.0

In [14]:
pitcher16=pitcher16.iloc[:,1:]

In [15]:
scaler_va = MinMaxScaler(feature_range=(0, 1))
signal_data = scaler_va.fit_transform(pitcher16)

In [16]:
signal_data.shape

(144, 5)

In [17]:
def multivariate_data(dataset, target, start_index, end_index, history_size, target_size, step, single_step=False):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        data.append(dataset[indices])

        if single_step:
            labels.append(target[i + target_size])
        else:
            labels.append(target[i:i + target_size])
    return np.array(data), np.array(labels)

In [19]:
look_back = 10
future_target = 0
dataset =signal_data
TRAIN_SPLIT=120
STEP=1
x_train_single, y_train_single = multivariate_data(dataset, dataset[:, 1], 0, TRAIN_SPLIT, look_back,
                                                   future_target,STEP, single_step=True)
x_val_single, y_val_single = multivariate_data(dataset, dataset[:, 1], TRAIN_SPLIT, None, look_back,
                                               future_target,STEP, single_step=True)

print('Single window of past history : {}'.format(x_train_single[0].shape))

Single window of past history : (10, 5)


In [22]:
import tensorflow as tf
BUFFER_SIZE=10
BATCH_SIZE =3
train_data_single = tf.data.Dataset.from_tensor_slices((x_train_single, y_train_single))
train_data_single = train_data_single.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()

val_data_single = tf.data.Dataset.from_tensor_slices((x_val_single, y_val_single))
val_data_single = val_data_single.batch(BATCH_SIZE).repeat()

single_step_model = tf.keras.models.Sequential()
single_step_model.add(tf.keras.layers.LSTM(32, input_shape=x_train_single.shape[-2:]))
single_step_model.add(tf.keras.layers.Dense(1))
single_step_model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='mae')


EPOCHS=50
EVALUATION_INTERVAL=20
early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)
single_step_history = single_step_model.fit(train_data_single, epochs=EPOCHS,
                                            steps_per_epoch =EVALUATION_INTERVAL)



W0913 21:43:37.162569 11860 deprecation.py:506] From F:\sda\envs\dlstudy\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/50
20/20 [==============================] - 1s 41ms/step - loss: 0.1746
Epoch 2/50
20/20 [==============================] - 0s 7ms/step - loss: 0.2025
Epoch 3/50
20/20 [==============================] - 0s 6ms/step - loss: 0.1779
Epoch 4/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1753
Epoch 5/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1841
Epoch 6/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1969
Epoch 7/50
20/20 [==============================] - 0s 6ms/step - loss: 0.1803
Epoch 8/50
20/20 [==============================] - 0s 6ms/step - loss: 0.1702
Epoch 9/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1959
Epoch 10/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1889
Epoch 11/50
20/20 [==============================] - 0s 7ms/step - loss: 0.1758
Epoch 12/50
20/20 [==============================] - 0s 6ms/step - loss: 0.1886
Epoch 13/50
20/20 [=============================

In [24]:
single_step_model.evaluate(x_val_single, y_val_single)

14/14 [==============================] - 0s 8ms/sample - loss: 0.1600


0.159995898604393

In [25]:
pre = single_step_model.predict(x_val_single)

In [26]:
scaler_va.inverse_transform(pre)

ValueError: non-broadcastable output operand with shape (14,1) doesn't match the broadcast shape (14,5)

In [27]:
x_val_single.shape

(14, 10, 5)

In [ ]:
y_val_single.shape

In [ ]:
x_val_single[1]

In [31]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM,Dropout
from keras.callbacks import EarlyStopping
look_back=10
model = Sequential()
model.add(LSTM(20, input_shape=x_train_single.shape[-2:])) #(timestep,feature)
model.add(Dropout(0.1))
model.add(Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam')

early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

model.fit(x_train_single, y_train_single, epochs=100,
          batch_size=5, verbose=1, callbacks=[early_stop])

Epoch 1/100
110/110 [==============================] - 1s 9ms/step - loss: 0.1208
Epoch 2/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0573
Epoch 3/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0469
Epoch 4/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0523
Epoch 5/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0523
Epoch 6/100
110/110 [==============================] - 0s 1ms/step - loss: 0.0479
Epoch 00006: early stopping


In [32]:
model.evaluate(x_val_single, y_val_single)

14/14 [==============================] - 0s 12ms/step


0.0453268364071846

In [53]:
x_val_single

array([[[0.88888889, 1.        , 0.65035361, 0.33333333, 0.8125    ],
        [0.36923077, 0.28081042, 0.81518152, 0.4       , 0.3125    ],
        [0.34188034, 0.1316932 , 0.48910891, 0.26666667, 0.3125    ],
        [0.06837607, 0.31693198, 0.13437058, 0.46666667, 0.0625    ],
        [0.        , 0.18958032, 0.4439604 , 0.4       , 0.        ],
        [0.35897436, 0.04775687, 0.49594959, 0.8       , 0.4375    ],
        [0.13675214, 0.38639653, 0.39444267, 0.06666667, 0.375     ],
        [0.34188034, 0.1316932 , 0.61290323, 0.26666667, 0.3125    ],
        [0.68376068, 0.74529667, 0.61810467, 0.33333333, 0.625     ],
        [0.17307692, 0.3288712 , 0.39444267, 0.33333333, 0.1875    ]],

       [[0.36923077, 0.28081042, 0.81518152, 0.4       , 0.3125    ],
        [0.34188034, 0.1316932 , 0.48910891, 0.26666667, 0.3125    ],
        [0.06837607, 0.31693198, 0.13437058, 0.46666667, 0.0625    ],
        [0.        , 0.18958032, 0.4439604 , 0.4       , 0.        ],
        [0.3589743

In [60]:
x_val_single[0][0]
x_val_single[1][1]

array([0.34188034, 0.1316932 , 0.48910891, 0.26666667, 0.3125    ])

In [46]:
x_val_single.shape

(14, 10, 5)